In [2]:
import numpy as np
from scipy import optimize
import scipy.optimize as opt
from scipy.optimize import minimize
import sympy as sp
import matplotlib.pyplot as plt
import sympy as sp
from ipywidgets import interact, FloatSlider




# The Solow Model
The Solow Model is a cornerstone macroeconomic framework that analyzes the determinants of long-term economic growth. This model focuses on how different factors such as capital accumulation, labor force characteristics, and technological advancements contribute to the output of an economy.

## Assumptions of the Solow Model

The model simplifies the complexity of real economies by making several key assumptions:

- **Single Representative Firm**: The economy consists of a single, representative firm.
- **Fixed Labor Force**: The total number of workers (labor force) is constant.
- **Constant Returns to Scale**: The production function is homogeneous of degree one, implying that scaling all inputs by a constant factor results in output scaling by the same factor.
- **Diminishing Returns to Capital**: As capital stock increases while other factors remain fixed, each additional unit of capital produces less additional output.
- **Closed Economy**: The economy does not engage in trade, meaning there are no imports or exports.

## Key Variables and Parameters

- $ Y $ = Output, commonly referred to as Gross Domestic Product (GDP).
- $ K $ = Capital stock, the total accumulation of produced means of production.
- $ L $ = Labor force, the total number of workers available for production.
- $ A $ = Technological progress, also known as total factor productivity (TFP).
- $ s $ = Savings rate, defined as the proportion of output that is saved and reinvested.
- $ \delta $ = Depreciation rate, the rate at which capital stock depreciates.
- $ n $ = Growth rate of the labor force.
- $ \alpha $ (alpha) = Output elasticity of capital, representing capital's contribution to output.
- $ 1 - \alpha $ = Output elasticity of labor, representing labor's contribution to output.

## Mathematical Representation of the Production Function

The production function can be expressed as:

$$
Y = A K^\alpha L^{1-\alpha}
$$

This function shows how output (Y) is determined by the levels of capital (K), labor (L), and technological progress (A). The parameters \(\alpha\) and \(1 - \alpha\) represent the output elasticities of capital and labor, respectively, indicating how sensitive output is to changes in each input.

## Dynamic Equations of the Model

The model's dynamics are driven by the equations of capital accumulation and technological growth:

### Capital Accumulation

$$
\frac{dK}{dt} = sY - \delta K
$$

This differential equation describes how the capital stock evolves over time, where $sY$ is the total investment from saving a fraction $s$ of output, and $\delta K$ is the loss of capital due to depreciation.

### Technological Progress

Assuming exogenous technological progress, the growth rate of technology ($g$) is constant:

$$
\frac{dA}{dt} = gA
$$

The model can be further analyzed to derive steady-state solutions and to explore the impact of changes in parameters like the savings rate, depreciation rate, and technology growth rate on the long-term economic growth of the economy.


In [3]:
# Define symbols more descriptively
Capital, Output, Labor, Technology = sp.symbols('Capital Output Labor Technology')
SavingsRate, DepreciationRate, LaborGrowthRate, TechGrowthRate, OutputElasticity = sp.symbols('SavingsRate DepreciationRate LaborGrowthRate TechGrowthRate OutputElasticity')

# Production Function
production_function = Technology * Capital**OutputElasticity * Labor**(1 - OutputElasticity)

# Capital Accumulation
capital_accumulation = SavingsRate * Output + Capital * (1 - DepreciationRate)

# Labor Force Growth (not directly used but defined for completeness)
labor_force_growth = LaborGrowthRate * Labor

# Technological Progress (not directly used but defined for completeness)
technological_progress = TechGrowthRate * Technology

# Output and Capital per Capita
output_per_capita = Output / Labor
capital_per_capita = Capital / Labor

# Output per Capita Growth
output_per_capita_growth = (1 - LaborGrowthRate) * (SavingsRate / capital_per_capita) - (DepreciationRate + TechGrowthRate)

# Solve for steady-state capital per capita
steady_state_equation = sp.solve(output_per_capita_growth, Capital)[0]
steady_state_function = sp.lambdify((SavingsRate, DepreciationRate, LaborGrowthRate, TechGrowthRate, OutputElasticity, Labor), steady_state_equation)

# Parameters for initial conditions and rates
params = {
    'Technology_initial': sp.symbols('Technology_initial'),
    'savings_rate': sp.symbols('savings_rate'),
    'depreciation_rate': sp.symbols('depreciation_rate'),
    'labor_force_growth_rate': sp.symbols('labor_force_growth_rate'),
    'technological_growth_rate': sp.symbols('technological_growth_rate'),
    'output_elasticity_value': sp.symbols('output_elasticity_value')
}

# Calculate steady-state values
steady_state_Capital = steady_state_function(params['savings_rate'], params['depreciation_rate'],
                                             params['labor_force_growth_rate'], params['technological_growth_rate'],
                                             params['output_elasticity_value'], 1)  # Assuming Labor = 1 for per capita calculations

# Substitute all values at once
steady_state_Output = production_function.subs({Capital: steady_state_Capital, Labor: 1, Technology: params['Technology_initial'], OutputElasticity: params['output_elasticity_value']})

print("Steady-State Capital per Capita =", steady_state_Capital)
print("Steady-State Output per Capita =", steady_state_Output)


Steady-State Capital per Capita = savings_rate*(1 - labor_force_growth_rate)/(depreciation_rate + technological_growth_rate)
Steady-State Output per Capita = Technology_initial*(savings_rate*(1 - labor_force_growth_rate)/(depreciation_rate + technological_growth_rate))**output_elasticity_value


## Steady-State 

In the Solow growth model, the steady-state represents a condition where the capital stock per capita remains constant over time, implying that the economy has reached an equilibrium level of capital that neither increases nor decreases.

### Steady-State Capital per Capita

The steady-state capital per capita is given by the following formula:

$$ k^* = \frac{s(1 - n)}{\delta + g} $$

Where:
- $k^* $ = steady-state capital per capita
- $ s $ = savings rate
- $ n $ = labor force growth rate
- $ \delta $ = depreciation rate of capital
- $ g $ = rate of technological progress

#### Economic Interpretation:
This equation shows that the steady-state capital per capita depends inversely on the sum of the depreciation rate and the technological progress rate. A higher savings rate or a lower labor growth rate leads to higher steady-state capital per capita, ceteris paribus. The denominator, \( \delta + g \), indicates that faster depreciation of capital or faster technological progress both require higher savings to maintain the same level of capital per capita, as new technology might make existing capital obsolete faster.

### Steady-State Output per Capita

Given the steady-state capital per capita, the steady-state output per capita can be derived as:

$$ y^* = A \left(\frac{s(1 - n)}{\delta + g}\right)^\alpha $$

Where:
- $ y^* $ = steady-state output per capita
- $ A $ = initial level of technology
- $ \alpha $ = output elasticity of capital, representing how output increases with an increase in capital

#### Economic Interpretation:
This formula demonstrates how the level of technology amplifies the effects of capital per capita on output per capita. The term $\left(\frac{s(1 - n)}{\delta + g}\right)^\alpha$ shows that output per capita is more responsive to changes in the capital-labor ratio in economies where capital is a more significant factor in production ($\alpha$ is large). Moreover, higher technological levels (represented by $A$) directly increase output per capita by making each unit of capital more productive.


In [11]:
# Initial settings and parameters for optimization
initial_tech = 1.0  # Initial level of technology
s_rate = 0.3  # Savings rate
d_rate = 0.05  # Depreciation rate
l_growth_rate = 0.02  # Labor force growth rate
t_growth_rate = 0.01  # Technological growth rate
cobb_douglas_exponent = 0.3  # Exponent in Cobb-Douglas production function

# Objective function to maximize (negative of output per capita for minimization)
def objective_function(capital_per_capita):
    """ Calculates the negative of the output per capita for maximization through minimization. """
    return -(initial_tech * capital_per_capita[0] ** cobb_douglas_exponent * (1 ** (1 - cobb_douglas_exponent)))

# Initial guesses for capital per capita optimization
initial_guesses = [0.1, 1.0, 10.0, 100.0, 1000.0]

# Perform optimization for each initial guess
for index, initial_capital in enumerate(initial_guesses):
    result = minimize(objective_function, [initial_capital], method='BFGS')
    
    if result.success:
        optimal_capital = result.x[0]
        optimal_output = initial_tech * (optimal_capital ** cobb_douglas_exponent) * (1 ** (1 - cobb_douglas_exponent))
        
        print(f"Test {index + 1}: Convergence successful!")
        print(f"per capita Optimal capital allocation : {optimal_capital:.4f}")
        print(f"Per capita optimal output allocation : {optimal_output:.4f}")
    else:
        print(f"Test {index + 1}: Convergence failed.")



Test 1: Convergence successful!
per capita Optimal capital allocation : 4276327.8011
Per capita optimal output allocation : 97.5711
Test 2: Convergence successful!
per capita Optimal capital allocation : 2728985.0621
Per capita optimal output allocation : 85.2708
Test 3: Convergence successful!
per capita Optimal capital allocation : 3292253.9188
Per capita optimal output allocation : 90.2086
Test 4: Convergence successful!
per capita Optimal capital allocation : 4216029.5784
Per capita optimal output allocation : 97.1563
Test 5: Convergence successful!
per capita Optimal capital allocation : 4634011.1129
Per capita optimal output allocation : 99.9510


# Economic Insights from Optimization Results

## Overview
This analysis optimizes capital per capita in a theoretical model using a Cobb-Douglas production function. Results indicate varied optimal capital levels leading to different outputs per capita based on initial capital estimates.

## Key Findings
1. **Initial Conditions**: The dependency on initial guesses for capital per capita suggests the presence of multiple local optima or a non-convex optimization landscape. This implies different economic outcomes based on starting conditions.
2. **Scale of Capital**: The large optimal capital values suggest a highly capital-intensive scenario. It highlights the importance of capital in the economy but also indicates diminishing returns as capital increases.
3. **Output Elasticity of Capital**: The elasticity parameter (0.3) shows that while capital significantly impacts output, it does not solely drive economic productivity. Other factors like labor or technological advancements are also crucial.
4. **Policy Implications**: The results can inform policies focusing on balanced growth—promoting capital investment while enhancing other factors like technology and labor efficiency.


In [12]:
# Base parameters
initial_technology = 1.0  # Initial level of technology
constant_savings_rate = 0.3  # Constant savings rate
annual_depreciation_rate = 0.05  # Annual depreciation rate
labor_growth_annual_rate = 0.02  # Annual labor force growth rate
technological_growth_rate = 0.01  # Annual technological growth rate
cobb_douglas_exponent = 0.3  # Exponent in Cobb-Douglas production function

# Function to calculate steady state capital and output
def calculate_steady_state(technology, savings_rate, depreciation_rate, labor_growth_rate, tech_growth_rate, output_elasticity):
    """Calculate steady-state capital and output for given economic parameters."""
    def objective_function(capital):
        return -(technology * (capital[0] ** output_elasticity) * (1 ** (1 - output_elasticity)))  # Minimization of negative output

    result = minimize(objective_function, [1])  # Initial guess for capital
    if result.success:
        capital_ss = result.x[0]
        output_ss = technology * (capital_ss ** output_elasticity) * (1 ** (1 - output_elasticity))
        return capital_ss, output_ss
    else:
        return None, None

# Testing parameter sensitivity
parameters = {
    'technology': [1.0, 1.5, 2.0],
    'savings_rate': [0.3, 0.4, 0.5],
    'depreciation_rate': [0.05, 0.06, 0.07],
    'labor_growth_rate': [0.02, 0.03, 0.04],
    'tech_growth_rate': [0.01, 0.02, 0.03],
    'output_elasticity': [0.3, 0.4, 0.5]
}

# Loop over different parameters to find steady-state values
for key, values in parameters.items():
    print(f"Testing changes in parameter: {key}")
    for value in values:
        # Update the parameter under test
        if key == 'technology':
            initial_technology = value
        elif key == 'savings_rate':
            constant_savings_rate = value
        elif key == 'depreciation_rate':
            annual_depreciation_rate = value
        elif key == 'labor_growth_rate':
            labor_growth_annual_rate = value
        elif key == 'tech_growth_rate':
            technological_growth_rate = value
        elif key == 'output_elasticity':
            cobb_douglas_exponent = value

        # Calculate steady-state values
        k_steadystate, y_steadystate = calculate_steady_state(initial_technology, constant_savings_rate, annual_depreciation_rate, labor_growth_annual_rate, technological_growth_rate, cobb_douglas_exponent)
        
        # Output results
        if k_steadystate is not None and y_steadystate is not None:
            print(f"{key} = {value:.2f} => Steady state: Capital = {k_steadystate:.4f}, Output = {y_steadystate:.4f}")
        else:
            print(f"{key} = {value:.2f} => Failed to calculate steady state.")

    print()  # New line for better separation between parameter tests

Testing changes in parameter: technology
technology = 1.00 => Steady state: Capital = 2728985.0621, Output = 85.2708
technology = 1.50 => Steady state: Capital = 6745948.7811, Output = 167.8047
technology = 2.00 => Steady state: Capital = 7919647.2507, Output = 234.7695

Testing changes in parameter: savings_rate
savings_rate = 0.30 => Steady state: Capital = 7919647.2507, Output = 234.7695
savings_rate = 0.40 => Steady state: Capital = 7919647.2507, Output = 234.7695
savings_rate = 0.50 => Steady state: Capital = 7919647.2507, Output = 234.7695

Testing changes in parameter: depreciation_rate
depreciation_rate = 0.05 => Steady state: Capital = 7919647.2507, Output = 234.7695
depreciation_rate = 0.06 => Steady state: Capital = 7919647.2507, Output = 234.7695
depreciation_rate = 0.07 => Steady state: Capital = 7919647.2507, Output = 234.7695

Testing changes in parameter: labor_growth_rate
labor_growth_rate = 0.02 => Steady state: Capital = 7919647.2507, Output = 234.7695
labor_growth_r


This analysis investigates how changes in key economic parameters affect the steady-state values of capital and output in a modeled economy. These parameters include technology level, savings rate, depreciation rate, labor growth rate, tech growth rate, and output elasticity.

## Findings

### 1. Technology
- **Impact**: Increasing technology from 1.00 to 2.00 leads to substantial increases in both capital and output. For example, technology at 1.50 results in a capital of 6,745,948.78 and an output of 167.80, while at 2.00, these values rise to 7,919,647.25 and 234.77 respectively.
- **Economic Interpretation**: Higher technology levels directly enhance the productivity of capital and labor, leading to higher output. This reflects the crucial role of technological advancements in economic growth.

### 2. Savings Rate
- **Impact**: Changes in the savings rate from 0.30 to 0.50 do not affect the steady-state values of capital and output, all remaining at 7,919,647.25 for capital and 234.77 for output.
- **Economic Interpretation**: The invariance of steady-state values despite changes in the savings rate suggests that the model might be assuming a perfectly elastic supply of savings or that other factors, such as output elasticity and technology, are dominating the effects of savings changes.

### 3. Depreciation Rate
- **Impact**: Variations in the depreciation rate from 0.05 to 0.07 show no change in the steady-state outcomes.
- **Economic Interpretation**: The stability of capital and output in response to changes in depreciation rate indicates that the economy's growth is not sensitive to the rate at which capital depreciates, possibly due to a compensating factor in the production function.

### 4. Labor Growth Rate
- **Impact**: Adjusting the labor growth rate from 0.02 to 0.04 also does not alter the steady-state results.
- **Economic Interpretation**: Similar to depreciation, the unchanging steady-state results suggest that labor growth does not critically impact the long-term capital and output levels, likely due to the fixed proportion of output elasticity in this model.

### 5. Technological Growth Rate
- **Impact**: No change in steady-state values with tech growth rate variations.
- **Economic Interpretation**: This indicates that short-term changes in technology growth rates do not significantly influence the long-term equilibrium of the economy within this model framework.

### 6. Output Elasticity
- **Impact**: Significant changes are observed when varying output elasticity. Increasing from 0.30 to 0.50 causes exponential increases in both capital and output.
- **Economic Interpretation**: Output elasticity represents how responsive the output is to changes in capital. Higher elasticity values mean that output is more sensitive to capital increases, leading to much larger outputs as capital increases. This highlights the pivotal role of capital's productivity in driving economic growth.



In [10]:

# Function to calculate steady-state capital and output
def calculate_steady_state(A, s, delta, n, g, a):
    def objective_function(k):
        return -(A * k[0] ** a * (1 ** (1 - a)))  # Minimization of negative output

    result = minimize(objective_function, [1])  # Initial guess for capital
    if result.success:
        k_ss = result.x[0]
        y_ss = A * (k_ss ** a) * (1 ** (1 - a))
        return k_ss, y_ss
    else:
        return None, None

# Function to get steady-state output based on parameters
def calculate_steady_state_output(A, s, delta, n, g, a):
    k_ss, y_ss = calculate_steady_state(A, s, delta, n, g, a)
    return y_ss if k_ss is not None and y_ss is not None else None

# Interactive visualization function
def interactive_plot(A=1.0, s=0.3, delta=0.05, n=0.02, g=0.01, a=0.3):
    param_ranges = {
        'A': np.linspace(0.8, 1.2, 5),
        's': np.linspace(0.2, 0.4, 5),
        'delta': np.linspace(0.02, 0.06, 5),
        'n': np.linspace(0.01, 0.03, 5),
        'g': np.linspace(0.005, 0.015, 5),
        'a': np.linspace(0.1, 0.5, 5)
    }
    fig, axs = plt.subplots(3, 2, figsize=(15, 15))
    fig.subplots_adjust(hspace=0.5, wspace=0.3)

    for ax, (key, values) in zip(axs.flatten(), param_ranges.items()):
        y_values = [calculate_steady_state_output(**{**{'A': A, 's': s, 'delta': delta, 'n': n, 'g': g, 'a': a}, **{key: v}}) for v in values]
        ax.plot(values, y_values)
        ax.set_title(f'Effect of {key} on Steady-State Output')
        ax.set_xlabel(f'{key}')
        ax.set_ylabel('Steady-State Output')
        ax.grid(True)
        
    plt.show()

# Include this line to ensure plots are shown directly within the Jupyter Notebook
%matplotlib inline

interact(interactive_plot,
         A=FloatSlider(value=1.0, min=0.8, max=1.2, step=0.05, description='Technology Level'),
         s=FloatSlider(value=0.3, min=0.2, max=0.4, step=0.05, description='Savings Rate'),
         delta=FloatSlider(value=0.05, min=0.02, max=0.06, step=0.01, description='Depreciation Rate'),
         n=FloatSlider(value=0.02, min=0.01, max=0.03, step=0.005, description='Labor Growth Rate'),
         g=FloatSlider(value=0.01, min=0.005, max=0.025, step=0.0025, description='Tech Growth Rate'),
         a=FloatSlider(value=0.3, min=0.1, max=0.5, step=0.05, description='Output Elasticity'))

interactive(children=(FloatSlider(value=1.0, description='Technology Level', max=1.2, min=0.8, step=0.05), Flo…

<function __main__.interactive_plot(A=1.0, s=0.3, delta=0.05, n=0.02, g=0.01, a=0.3)>

# Extension to the Solow Model and Graph Analysis

## Graph Analysis and Economic Implications

### Key Observations:
1. **Technology Level (A)**: Steady-state output increases linearly with technology, underscoring technology's pivotal role in enhancing productivity.
2. **Savings Rate (s)** and **Depreciation Rate (δ)**: Output remains unchanged across different rates, suggesting other model parameters, such as technology, dominate these effects.
3. **Labor and Technological Growth Rates (n, g)**: Output is invariant to changes, indicating that short-term adjustments in these rates do not affect long-term output.
4. **Output Elasticity of Capital (α)**: Shows a pronounced exponential increase in output with higher elasticity, highlighting the significant impact of capital productivity on economic growth.


### Conclusion

By making technology endogenous and examining different growth paths, this extended Solow model offers a more nuanced understanding of the factors driving economic growth and their interdependencies. This approach not only aligns with empirical observations but also enhances the model's utility for policy analysis.


In [7]:
initial_technology_level = 1.0  # Initial level of technology
constant_savings_rate = 0.3  # Savings rate
annual_depreciation_rate = 0.05  # Depreciation rate
labor_growth_annual_rate = 0.02  # Labor force growth rate

# Function to calculate steady state level of capital based on technology changes
def calculate_steady_state_capital(technology_level):
    """ Calculate steady state capital based on technology level, considering economic factors.
    
    Args:
        technology_level (float): Current technology level.

    Returns:
        float: Steady state capital.
    """
    steady_state_capital = ((constant_savings_rate * technology_level) /
                            (annual_depreciation_rate + labor_growth_annual_rate * technology_level)) ** (1 / (1 - labor_growth_annual_rate))
    return steady_state_capital

# Function to visualize the impact of technological growth on steady state capital over time
def visualize_steady_state_growth(tech_growth_rate):
    """ Plot the steady state capital over time based on the technology growth rate.

    Args:
        tech_growth_rate (float): Annual technological growth rate.
    """
    time_periods = np.arange(0, 500)  # Time period for simulation, from 0 to 500 years
    technology_growth = initial_technology_level * np.exp(tech_growth_rate * time_periods)
    
    steady_state_capital_over_time = calculate_steady_state_capital(technology_growth)
    
    plt.figure(figsize=(10, 6))
    plt.plot(time_periods, steady_state_capital_over_time, label=f'Technological Growth Rate: {tech_growth_rate:.2f}')
    plt.xlabel('Time (years)')
    plt.ylabel('Steady State Capital (k)')
    plt.title(f'Steady State Capital for Technological Growth Rate: {tech_growth_rate:.2%}')
    plt.legend()
    plt.grid(True)
    plt.show()

# Interactive sliders to adjust and visualize the effects of different technological growth rates
interact(visualize_steady_state_growth, tech_growth_rate=FloatSlider(value=0.01, min=0.00, max=0.10, step=0.01, description='Technological Growth Rate'))

interactive(children=(FloatSlider(value=0.01, description='Technological Growth Rate', max=0.1, step=0.01), Ou…

<function __main__.visualize_steady_state_growth(tech_growth_rate)>

# Conclusion


### Effects of Standard Economic Parameters:
- **Savings Rate (s), Depreciation Rate (δ), and Labor Force Growth Rate (n)**:
  The steady-state levels of capital and output per capita are invariant to changes in the savings rate, depreciation rate, and labor force growth rate. Adjustments in these parameters affect the speed of convergence rather than the long-term equilibrium levels. For instance, a higher savings rate accelerates the accumulation of capital, thus shortening the time required to reach the steady state.

### Influence of Productivity and Capital Elasticity:
- **Total Factor Productivity (A) and Output Elasticity of Capital (α)**:
  Increasing total factor productivity or the output elasticity of capital elevates the steady state levels of both capital and output per capita. A higher total factor productivity enhances the effectiveness of both labor and capital in producing output, thereby raising the output for any given level of capital and labor. Similarly, an increase in the output elasticity of capital means that output becomes more sensitive to changes in the capital stock, leading to higher output levels at equilibrium.

## Extension Involving Technological Growth Rate:
- **Impact of Technological Growth Rate (g)**:
  Our extended analysis includes adjusting the growth rate of technology, as depicted in the provided graph. Increasing the technological growth rate enhances the economy's ability to reach its steady state faster. This is shown by the steep initial increase in steady-state capital as the technological growth rate improves. Although higher growth rates accelerate convergence, the ultimate steady-state levels remain unaffected. Conversely, a lower technological growth rate results in a slower approach to the steady state, prolonging the period of economic adjustment.

